This is a sample for celestial navigation for a stationary observer<br>
© August Linnman, 2025, email: august@linnman.net<br>
MIT License (see [LICENSE file](LICENSE))

Jupyter notebook for navigation on a moving ship/vehicle. 

In [1]:
# Importing Python libraries
from time import time
from starfix import Sight, SightTrip, get_representation, LatLonGeodetic,\
                    IntersectError

In [2]:
# Estimated (DR) position at start
s1LatLon = LatLonGeodetic (59, 18)
# Estimated course and speed
C_COURSE = 175
SPEED = 20

#This is the star fix for s1, the starting point

Sight.set_estimated_position (s1LatLon)

s1 = Sight (  object_name          = "Sun",
              set_time             = "2024-06-20 06:14:38+00:00",
              measured_alt         = "30:51:27.1"
              )

In [3]:
# Star fix for ending point

s2 = Sight (  object_name          = "Sun",
              set_time             = "2024-06-20 07:14:38+00:00",
              measured_alt         = "38:34:21.6"
              )

In [ ]:
# Sight reduction

st = SightTrip (sight_start               = s1,\
                 sight_end                = s2,\
                 estimated_starting_point = s1LatLon,\
                 course_degrees           = C_COURSE,\
                 speed_knots              = SPEED)
m = None
try:
    intersections, _, _ = st.get_intersections (return_geodetic=True)
    assert isinstance (intersections, tuple)
    print ("Starting point = " + str(get_representation(intersections[1],1)))
    print ("End point = " + str(get_representation(intersections[0],1)))
    m = st.render_folium(intersections)
except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve)) 
m